# Management of Modeling Data

In [18]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3

## Pandas DataFrames

In [7]:
df = pd.read_csv("mutation.csv")

In [8]:
df.head()

,source,position,initial_nt,changed_nt,effect,type,aa_change,gene_id,ggene_id,is_an_mutation,gene_position,mutation,key_mutation
0,Chromosome,412,CCCCCCTCGCAGCCCCC,CCCCCC,HIGH,NONE,SQPP90,DVU0001,DVU0001,False,DVU0001.412,FRAME_SHIFT,DVU0001.412.CCCCCCTCGCAGCCCCC.CCCCCC
1,Chromosome,42867,AG,AGG,MODIFIER,NONE,NaN,NaN,DVH__.42867,True,DVH__.42867,INTERGENIC,DVH__.42867.AG.AGG
2,Chromosome,211389,T,TA,MODIFIER,NONE,NaN,NaN,DVH__.211389,True,DVH__.211389,INTERGENIC,DVH__.211389.T.TA
3,Chromosome,882511,GC,G,MODIFIER,NONE,NaN,NaN,DVH__.882511,True,DVH__.882511,INTERGENIC,DVH__.882511.GC.G
4,Chromosome,885161,TGGAGGAG,TGGAG,MODERATE,NONE,YS204Y,DVU0799,DVU0799,False,DVU0799.885161,CODON_DELETION,DVU0799.885161.TGGAGGAG.TGGAG


In [10]:
# Extract a column from the table
gene_ids = df['gene_id']
len(gene_ids)

587

In [11]:
# Extract unique values from the table
gene_ids = df['gene_id'].unique()
len(gene_ids)

229

In [13]:
# Obtain the culture data. species_mix describes the community present
#  L=line, E=EPD, B=coculture, D=DVH monoculture, M=MMP monoculture
df = pd.read_csv("culture.csv")
df.head()

,rate,yield,key_culture,species_mix
0,0.013996,0.419886,C1,L
1,0.014905,0.448689,C2,L
2,0.013935,0.416881,C3,L
3,0.013353,0.428859,C4,L
4,0.014579,0.446349,C5,L


In [20]:
# Create a new dataframe that consists of just co-cultures
df_coculture = df[df['species_mix'] == 'B']
df_coculture.head()

,rate,yield,key_culture,species_mix
151,0.035581,0.531100,C152,B
152,0.035072,0.528433,C153,B
153,0.035883,0.534548,C154,B
154,0.027380,0.485459,C155,B
155,0.024985,0.502712,C156,B


In [21]:
# Standardize values
def standardize(df, col):
    mean = np.mean(df[col])
    std = np.std(df[col])
    new_df = df.copy()
    new_df[col] = (df[col]-mean)/std
    return new_df
df_new = standardize(df_coculture, 'rate')
df_new = standardize(df_new, 'yield')
df_new.head()

,rate,yield,key_culture,species_mix
151,1.395037,1.181453,C152,B
152,1.338781,1.147782,C153,B
153,1.428432,1.224982,C154,B
154,0.488368,0.605266,C155,B
155,0.223483,0.823077,C156,B


## Using SQL with DataFrames

In [26]:
conn = sqlite3.connect("example.db")

In [23]:
df_new.to_sql("standardized_culture", conn)


In [27]:
for filename in ["gene_description", "mutation"]:
    df = pd.read_csv("%s.csv" % filename)
    df.to_sql(filename, conn)

In [28]:
conn.commit()

In [29]:
conn.close()